In [100]:
import pandas as pd
import os, sys
import json
import pymysql

In [101]:
Resultados_modelos = pd.DataFrame()

In [102]:
models_comparaison = pd.DataFrame({'Model': ['Kmeans', 'Dbscan','LinearRegression', 'PolynomialFeatures','SVR','LogisticRegression','KNeighborsClassifier','SVC','DecisionTreeRegressor','RandomForestRegressor','DecisionTreeClassifier','RandomForestClassifier','Red neuronal'],
'Parameter' : ['None','None','n_job', 'degree','Kernel_C_gamma_epsilon','max_iter','n_neighbors','Kernel_C_gamma','max_depth','n_estimators_max_depth','None','n_splits_n_repeats','Dense_(activation_input_shape)'],
'Recall':['None', 'None','None', 'None','None', 'None','None','LogisticRegression','KNeighborsClassifier','SVC','DecisionTreeRegressor','RandomForestRegressor','RandomForestClassifier','DecisionTreeClassifier','Red neuronal'], 
'Score':['None', 'None','LinearRegression', 'PolynomialFeatures','SVR','LogisticRegression','KNeighborsClassifier','SVC','DecisionTreeRegressor','RandomForestRegressor','RandomForestClassifier','DecisionTreeClassifier','Red neuronal'],
'MAE':['None', 'None','LinearRegression', 'PolynomialFeatures','SVR','LogisticRegression','KNeighborsClassifier','SVC','DecisionTreeRegressor','RandomForestRegressor','RandomForestClassifier','DecisionTreeClassifier','Red neuronal'],
'Accuracy': ['None', 'None','LinearRegression', 'PolynomialFeatures','SVR','LogisticRegression','KNeighborsClassifier','SVC','DecisionTreeRegressor','RandomForestRegressor','RandomForestClassifier','DecisionTreeClassifier','Red neuronal']})


In [103]:
models_comparaison.to_csv ('../data/Bases_trabajo/models_comparaison.csv')

In [104]:
# datos para conectarme a MySQL

#"IP_DNS": "consciencesai.com",
#   "PORT": 30001,
#   "USER": Vuestro DNI (letra/s en minúscula),
#   "PASSWORD": el id de vuestro correo,
#   "BD_NAME":  el USER pero con "_ds_april_2021_db"al final 

In [105]:
import pymysql
from sqlalchemy import create_engine
from sqlalchemy.pool import NullPool

class MySQL:

    def __init__(self, IP_DNS, USER, PASSWORD, BD_NAME, PORT):
        self.IP_DNS = "consciencesai.com"
        self.USER = "13297422h"
        self.PASSWORD = "jorginagc"
        self.BD_NAME = "13297422h_ds_april_2021_db"
        self.PORT = 30001
        self.SQL_ALCHEMY = 'mysql+pymysql://' + self.USER + ':' + self.PASSWORD + '@' + self.IP_DNS + ':' + str(self.PORT) + '/' + self.BD_NAME
        
    def connect(self):
        # Open database connection
        self.db = pymysql.connect(host=self.IP_DNS,
                                  user=self.USER, 
                                  password=self.PASSWORD, 
                                  database=self.BD_NAME, 
                                  port=self.PORT)
        # prepare a cursor object using cursor() method
        self.cursor = self.db.cursor()
        print("Connected to MySQL server [" + self.BD_NAME + "]")
        return self.db

    def close(self):
        # disconnect from server
        self.db.close()
        print("Close connection with MySQL server [" + self.BD_NAME + "]")
    
    def execute_interactive_sql(self, sql, delete=False):
        """ NO SELECT """
        result = 0
        try:
            # Execute the SQL command
            self.cursor.execute(sql)
            # Commit your changes in the database
            self.db.commit()
            print("Executed \n\n" + str(sql) + "\n\n successfully")
            result = 1
        except Exception as error:
            print(error)
            # Rollback in case there is any error
            self.db.rollback()
        return result
        
    def execute_get_sql(self, sql):
        """SELECT"""
        results = None
        print("Executing:\n", sql)
        try:
            # Execute the SQL command
            self.cursor.execute(sql)
            # Fetch all the rows in a list of lists.
            results = self.cursor.fetchall()
        except Exception as error:
            print(error)
            print ("Error: unable to fetch data")
        
        return results

    def create_engine(self):
        engine = create_engine(self.SQL_ALCHEMY, poolclass=NullPool)
        conn = engine.connect()
        print("Connected to MySQL server [" + self.BD_NAME + "]")
        return conn, engine


    def execute_engine(self, conn, sql):
        result = 0
        try:
            result = conn.execute(sql).fetchall()
            print("Executed \n\n" + str(sql) + "\n\n successfully")
            
        except Exception as error:
            print(error)
        return result

    def close_engine(self, conn, engine):
        conn.close()
        engine.dispose()
        print("Close connection with MySQL server [" + self.BD_NAME + "]")

    def generate_insert_into_people_sql(self, to_insert):
        """
        This must be modified according to the table structure
        """
        nombre = to_insert[0]
        apellidos = to_insert[1]
        direccion = to_insert[2]
        edad = to_insert[3]
        nota = to_insert[4]
        
        sql = """INSERT INTO people
            (MOMENTO, NOMBRE, APELLIDOS, DIRECCION, EDAD, NOTA)
            VALUES
            (NOW(), '""" + nombre + """', '""" + apellidos + """', '""" + direccion + """', '""" + edad + """', '""" + nota + """')"""

        sql = sql.replace("\n", "").replace("            ", " ")
        return sql

In [106]:
mysql_db = MySQL(IP_DNS="consciencesai.com", USER="13297422h", PASSWORD="jorginagc", BD_NAME="13297422h_ds_april_2021_db", PORT=30001)
conn = mysql_db.connect()

# select_sql = 'SELECT * FROM models_comparaison'
# select_result = mysql_db.execute_get_sql(sql=select_sql)

# df = pd.read_sql('SELECT * FROM models_comparaison', con=conn)
# df

Connected to MySQL server [13297422h_ds_april_2021_db]


In [107]:
models_comparaison = pd.read_csv('../data/Bases_trabajo/models_comparaison.csv')

In [108]:
select_sql = 'SELECT * FROM models_comparaison'
select_result = mysql_db.execute_get_sql(sql=select_sql)

Executing:
 SELECT * FROM models_comparaison


In [109]:
df = pd.read_sql('SELECT * FROM models_comparaison', con=conn)
df

,Model,Parameter,Recall,Score,MAE,Accuracy
0,LinearRegression,n_job,None,LinearRegression,LinearRegression,LinearRegression
1,PolynomialFeatures,degree,None,PolynomialFeatures,PolynomialFeatures,PolynomialFeatures
2,SVR,Kernel_C_gamma_epsilon,None,SVR,SVR,SVR
3,LogisticRegression,max_iter,LogisticRegression,LogisticRegression,LogisticRegression,LogisticRegression
4,KNeighborsClassifier,n_neighbors,KNeighborsClassifier,KNeighborsClassifier,KNeighborsClassifier,KNeighborsClassifier
5,SVC,Kernel_C_gamma,SVC,SVC,SVC,SVC
6,max_depth,DecisionTreeRegressor,DecisionTreeRegressor,DecisionTreeRegressor,DecisionTreeRegressor,DecisionTreeRegressor
7,n_estimators_max_depth,RandomForestRegressor,RandomForestRegressor,RandomForestRegressor,RandomForestRegressor,RandomForestRegressor
8,RandomForestClassifier,n_splits_n_repeats,RandomForestClassifier,RandomForestClassifier,RandomForestClassifier,RandomForestClassifier
9,DecisionTreeClassifier,None,DecisionTreeClassifier,DecisionTreeClassifier,DecisionTreeClassifier,DecisionTreeClassifier


In [110]:
# mysql_db = MySQL(IP_DNS="consciencesai.com", USER="13297422h", PASSWORD="jorginagc", BD_NAME="13297422h_ds_april_2021_db", PORT=30001)
# conn,engine = mysql_db.create_engine()

In [111]:
# models_comparaison = pd.read_csv('../data/Bases_trabajo/models_comparaison.csv')

In [112]:
 #models_comparaison.to_sql (name = 'models_comparaison.csv', con= conn, if_exists='fail', index = False)

In [113]:
# select_sql = 'SELECT * FROM models_comparaison' 
#resultado = mysql_db.execute_engine(conn, select_sql)

# resultado

In [114]:
# df = pd.read_sql('SELECT * FROM models_comparaison_1', con=conn)
# df

In [115]:
mysql_db.close_engine(conn, engine)

Close connection with MySQL server [13297422h_ds_april_2021_db]
